In [11]:
import pandas as pd
import sqlite3
import requests

In [3]:
connection = sqlite3.connect("kino.sqlite")
cursor = connection.cursor()

##  Загрузка постеров через OMDb API
Для работы нужно получить ключ (бесплатный вариант разрешает 1000 запросов в день)
http://www.omdbapi.com/

In [ ]:
api_key = ''

In [6]:
def format_imdb_id(movie_id):
    movie_id = str(movie_id)
    id_len = len(movie_id)
    if id_len < 7:
        return ('0' * (7 - id_len)) + movie_id
    else:
        return movie_id

In [26]:
api_call_results = []
api_call_posters = []
for movie_id in movie_ids['movie_id']:
    req_url_head = 'http://www.omdbapi.com/?i=tt'
    req_url_tail = '&apikey='
    req_url = req_url_head + format_imdb_id(movie_id) + req_url_tail + api_key

    res = requests.get(req_url)
    api_call_results.append(res)
    api_call_posters.append(res.json()['Poster'])
    print(res)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [28]:
print(len(api_call_posters))

500


In [31]:
api_call_results[1].json()

{'Title': 'Sherlock Jr.',
 'Year': '1924',
 'Rated': 'Passed',
 'Released': '11 May 1924',
 'Runtime': '45 min',
 'Genre': 'Action, Comedy, Romance',
 'Director': 'Buster Keaton',
 'Writer': 'Jean C. Havez, Joseph A. Mitchell, Clyde Bruckman',
 'Actors': 'Buster Keaton, Kathryn McGuire, Joe Keaton',
 'Plot': "A film projectionist longs to be a detective, and puts his meagre skills to work when he is framed by a rival for stealing his girlfriend's father's pocketwatch.",
 'Language': 'English, None',
 'Country': 'United States',
 'Awards': '1 win',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BZWFhOGU5NDctY2Q3YS00Y2VlLWI1NzEtZmIwY2ZiZjY4OTA2XkEyXkFqcGdeQXVyMDI2NDg0NQ@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.2/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '92%'}],
 'Metascore': 'N/A',
 'imdbRating': '8.2',
 'imdbVotes': '49,981',
 'imdbID': 'tt0015324',
 'Type': 'movie',
 'DVD': '11 Dec 2012',
 'BoxOffice': '$977,375',
 'Production': 'N/A',


In [38]:
def get_int_from_imdb_id(imdb_id):
    return int(imdb_id[2::])

In [53]:
df = pd.DataFrame(columns=['id', 'poster url'])
for call in api_call_results:
    json = call.json()
    row = pd.DataFrame({'id': [get_int_from_imdb_id(json['imdbID'])], 'poster url': [json['Poster']]})
    df = pd.concat([row, df])

df = df.set_index('id')
df

,poster url
id,
15097216,https://m.media-amazon.com/images/M/MV5BOTM0NW...
14392248,https://m.media-amazon.com/images/M/MV5BNWRkMj...
13991232,https://m.media-amazon.com/images/M/MV5BNDZkZD...
13706018,https://m.media-amazon.com/images/M/MV5BMWM4Nj...
13675832,https://m.media-amazon.com/images/M/MV5BZTUzZj...
...,...
17925,https://m.media-amazon.com/images/M/MV5BYmRiMD...
17136,https://m.media-amazon.com/images/M/MV5BMTg5YW...
15864,https://m.media-amazon.com/images/M/MV5BZjEyOT...


In [62]:
for m_id, m_poster in df.itertuples():
    cursor.execute(f'''
     UPDATE
     movie
     SET
     movie_poster_url = '{m_poster}'
     WHERE
     movie.movie_id = {m_id}
    ''')

In [63]:
connection.commit()